In [34]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
def read_article(text):
    article = text.split(". ")
    sentences = []

    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))

    sentences.pop()

    return sentences

In [36]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [word.lower() for word in sent1]
    sent2 = [word.lower() for word in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for word in sent1:
        if word in stopwords:
            continue
        vector1[all_words.index(word)] += 1

    for word in sent2:
        if word in stopwords:
            continue
        vector2[all_words.index(word)] += 1

    return 1 - cosine_distance(vector1, vector2)

In [38]:
def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [39]:
def generate_summary(text, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    sentences =  read_article(text)
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    for i in range(min(top_n, len(ranked_sentence))):  # Check against the actual number of sentences
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    return ". ".join(summarize_text)

In [41]:
# Sample text for summarization
text = """
Having a best friend by Shaikh Subuhi is one of the best friendship moral stories in English.
The story is about two friends who were walking through the desert. During the journey, they
argued over something, and one friend slapped the other. The one who got slapped was hurt by
this gesture of his best friend but did not react. He quietly wrote in the sand, “Today my best friend slapped me.”
After some time, they found an oasis and started taking a bath in the lake. Suddenly, the one who had been slapped
started drowning. Then his friend came to his rescue and saved him. After he recovered from the drowning, he
engraved “Today my best friend saved my life” on a stone.
The friend who had slapped earlier and later saved his best friend asked, “After I slapped you, you wrote in the sand,
and now, as I saved you, you write on a stone, why?” The other friend replied, “I wrote in on sand because we should
not keep the feeling of getting hurt by someone for a long time. But, when someone does something good for us, we must
remember it forever like a message engraved on a stone that nothing can erase”.
Moral: Remember the good things that happen in life, not the bad memories.
"""

In [42]:
# Generate and print the summary
summary = generate_summary(text)
print(summary)

The one who got slapped was hurt by 
this gesture of his best friend but did not react. After he recovered from the drowning, he 
engraved “Today my best friend saved my life” on a stone.
The friend who had slapped earlier and later saved his best friend asked, “After I slapped you, you wrote in the sand,
and now, as I saved you, you write on a stone, why?” The other friend replied, “I wrote in on sand because we should
not keep the feeling of getting hurt by someone for a long time. During the journey, they 
argued over something, and one friend slapped the other. 
Having a best friend by Shaikh Subuhi is one of the best friendship moral stories in English. He quietly wrote in the sand, “Today my best friend slapped me.”
After some time, they found an oasis and started taking a bath in the lake
